In [7]:
import plotly.graph_objects as go

import pandas as pd
df = pd.read_csv('../resources/us_accidents_filtered_rows.csv')

df['Start_Time'] = pd.to_datetime(df['Start_Time'])

# Function to determine the season
def get_season(date):
    year = date.year
    seasons = {'spring': (pd.Timestamp(f'{year}-03-21'), pd.Timestamp(f'{year}-06-20')),
               'summer': (pd.Timestamp(f'{year}-06-21'), pd.Timestamp(f'{year}-09-22')),
               'autumn': (pd.Timestamp(f'{year}-09-23'), pd.Timestamp(f'{year}-12-20')),
               'winter': (pd.Timestamp(f'{year}-12-21'), pd.Timestamp(f'{year + 1}-03-20'))}

    for season, (start, end) in seasons.items():
        if start <= date <= end:
            return season
    return 'winter'  # dates outside the ranges fall in winter (from Dec 21 to Mar 20)

# Apply the function to create a new column 'season'
df['season'] = df['Start_Time'].apply(get_season)

In [8]:
state_counts = df['State'].value_counts().reset_index()
state_counts.columns = ['State', 'Count']

def get_state_counts_for_season(season):
    return df[df['season'] == season]['State'].value_counts().reset_index()

seasons = ['winter', 'spring', 'summer', 'autumn']
choropleths = []

for season in seasons:
    state_counts = get_state_counts_for_season(season)
    state_counts.columns = ['State', 'Count']
    choropleths.append(
        go.Choropleth(
            locations=state_counts['State'],
            z=state_counts['Count'],
            locationmode='USA-states',
            colorscale='Reds',
            colorbar_title="Number of Accidents",
            visible=False  # initially make all choropleths invisible
        )
    )

choropleths[0].visible = True

fig = go.Figure(data=choropleths)

updatemenu = [
    {
        'buttons': [
            {'label': 'Winter', 'method': 'update', 'args': [{'visible': [s == 'winter' for s in seasons]}, {'title': 'Amount of accidents in Winter'}]},
            {'label': 'Spring', 'method': 'update', 'args': [{'visible': [s == 'spring' for s in seasons]}, {'title': 'Amount of accidents in Spring'}]},
            {'label': 'Summer', 'method': 'update', 'args': [{'visible': [s == 'summer' for s in seasons]}, {'title': 'Amount of accidents in Summer'}]},
            {'label': 'Autumn', 'method': 'update', 'args': [{'visible': [s == 'autumn' for s in seasons]}, {'title': 'Amount of accidents in Autumn'}]},
        ],
        'direction': 'down',
        'showactive': True,
    }
]


fig.update_layout(
    title_text='Amount of accidents by State and Season',
    geo_scope='usa',  # limit map scope to USA
    updatemenus=updatemenu,
)

fig.show()